## Carga informacion

In [11]:
import glob
import os
import pandas as pd
import warnings
from pathlib import Path

# Ocultar warnings de openpyxl sobre headers/footers
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

# ==========
# 1) Unir los 8 Excels de septiembre
# ==========
ruta_ns = r"/home/jpcano/Donsson-Proyectos/INDICADOR NS/dataset/ns_septiembre"
archivos_ns = glob.glob(os.path.join(ruta_ns, "*.xlsx"))

if not archivos_ns:
    raise FileNotFoundError(f"No se encontró ningún archivo en {ruta_ns}")

# Unir todos los Excels en un solo DataFrame
df_unido = pd.concat([pd.read_excel(f) for f in archivos_ns], ignore_index=True)

print(f"✅ Se unieron {len(archivos_ns)} archivos de septiembre con {df_unido.shape[0]} filas totales")

# ==========
# 2) Cargar inventario limpio
# ==========
ruta_inventario = "/home/jpcano/Donsson-Proyectos/INVENTARIO/inventario_limpio.xlsx"

if not os.path.exists(ruta_inventario):
    raise FileNotFoundError(f"No existe el archivo {ruta_inventario}")

df_inventario = pd.read_excel(ruta_inventario)

print(f"📦 Inventario cargado con {df_inventario.shape[0]} filas desde: {ruta_inventario}")



✅ Se unieron 8 archivos de septiembre con 38816 filas totales
📦 Inventario cargado con 15217 filas desde: /home/jpcano/Donsson-Proyectos/INVENTARIO/inventario_limpio.xlsx


In [12]:
df_unido.sample(5)

,Sucursal,Marca,Clase,Producto,Descripcion,Enero nv,Febrero nv,Marzo nv,Abril nv,Mayo nv,...,Marzo vp,Abril vp,Mayo vp,Junio vp,Julio vp,Agosto vp,Septiembre vp,Octubre vp,Noviembre vp,Diciembre vp
18904,SUCURSAL VALLADOLID,DONSSON,C,DAR02349025,[DAR02349025] DA2349 FILTRO AIRE 1¦ HITACHI 42...,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1865,PRINCIPAL COTA,AUT*PARTS,C,DAC00041189,[DAC00041189] DAC041 FILTRO AIRE CABINA CAMION...,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3208,PRINCIPAL COTA,BALDWIN,C,BCS00429125,[BCS00429125] GS429 FILTRO COMBUSTIBLE ALTA EF...,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16226,SUCURSAL VALLADOLID,DONSSON,C,DAX91006222,[DAX91006222] GXPB006 PREFILTRO BAJOCAPOT 140C...,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35186,SUCURSAL CALLE 6,OTROS,C,DAE01096122,[DAE01096122] GA1096 FILTRO AIRE NISSAN MARCH ...,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df_sin_clase = df_unido[df_unido["Clase"]=="SIN CLASE X SUCURSAL"]  # identificar productos sin categoria y revisar si son nuevos

df_sin_clase = df_sin_clase.groupby("Descripcion")["Producto"].count()
df_sin_clase.sample(5)

Descripcion
[DAB28118025] DA8118KIT FILTRO AIRE JUEGO BUSES SCANIA SERIE K,IVECO,MAN,RENA (025 DA8118KIT)    8
[DLX00393020] GX393 FILTRO SEPARADOR AIRE / ACEITE - INGERSOLL RAND -  KV320-008  (020 GX393)    8
[ACUEDUCTO Y ALCANTARILLADO] ACUEDUCTO Y ALCANTARILLADO                                          8
[TAP8111] PARTE PLASTICA CUERPO Y TAPA DA8111 (TAPA DA8111)                                      8
[OTROS] OTROS COBROS EN SUMINISTRO DE FILTROS                                                    8
Name: Producto, dtype: int64

## Limpieza

In [14]:
from datetime import datetime
import re

def limpiar_y_transformar(df_unido):
    # 1. Obtener el mes actual en español manualmente
    meses_es = [
        "Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio",
        "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre"
    ]
    mes_actual = meses_es[datetime.now().month - 1]

    # 2. Quitar columnas desde el mes siguiente al actual
    meses_nv = [f"{mes.lower()} nv" for mes in meses_es]
    meses_vp = [f"{mes.lower()} vp" for mes in meses_es]

    idx_mes_actual = meses_es.index(mes_actual)

    # Columnas a eliminar: desde el mes siguiente hasta diciembre
    meses_a_borrar = meses_nv[idx_mes_actual+1:] + meses_vp[idx_mes_actual+1:]
    columnas_a_eliminar = [col for col in df_unido.columns if col.lower() in meses_a_borrar]
    df_unido = df_unido.drop(columns=columnas_a_eliminar, errors='ignore')

    # 3. Clasificación categoría
    marcas_filtros = [ "BALDWIN", "DONSSON", "AUT*PARTS", "RAMA", "RACOR USA", "RACOR BRASIL"
    ]

    #marcas_aceite_lubricantes = []

    def clasificar_categoria(marca):
        marca = str(marca).upper().strip()
        if marca == "OTROS":
            return "Otros"
        elif marca in marcas_filtros:
            return "Filtro"
        else:
            return "Aceite/Lubricante"

    df_unido["categoria_producto"] = df_unido["Marca"].apply(clasificar_categoria)



    # 4. Extraer código interno        
    def extraer_codigo(descripcion, categoria):
        if not isinstance(descripcion, str):
            return None

    # Para Aceite/Lubricante, tomar directamente lo que está dentro de los corchetes
        if categoria == "Aceite/Lubricante":
            match = re.search(r"\[([A-Z0-9\-]+)\]", descripcion)
            if match:
                return match.group(1)

    # Para Filtros, aplicar la lógica anterior

    
        elif categoria == "Filtro" or categoria == "Otros":
            match = re.search(r"\[([A-Z0-9\-]+)\]\s+(\S+)", descripcion)
            if match:
                codigo_base = match.group(1)
                palabra_siguiente = match.group(2)
            
            # Tomar los últimos 3 dígitos (no caracteres): buscar solo dígitos desde el final
            ultimos_digitos = ''.join(re.findall(r'\d', codigo_base))[-3:]
            return f"{ultimos_digitos} {palabra_siguiente}"

        

    df_unido["Codigo_interno"] = df_unido.apply(
    lambda row: extraer_codigo(row["Descripcion"], row["categoria_producto"]),
    axis=1)
    df_unido['Codigo_Numerico'] = df_unido['Codigo_interno'].str.split(' ').str[0]


    # 5. Definir las listas de códigos numéricos para filtros
    importacion = ['189', '125', '138', '137', '222', '225']
    nacional = ['004', '190', '005']
    fabricado = ['025']

    # 5. Crear una función de clasificación que considera la categoría y el código numérico
    def clasificar_producto(row):
        # Si la categoría es "Filtro", usamos las listas para clasificar
        if row['categoria_producto'] == 'Filtro':
            if row['Codigo_Numerico'] in importacion:
                return 'Importacion'
            elif row['Codigo_Numerico'] in nacional:
                return 'Nacional'
            elif row['Codigo_Numerico'] in fabricado:
                return 'Fabricado'
            else:
                return 'Desconocido' # Si es un filtro con un código no listado
            # Para cualquier otra categoría (Aceite/Lubricante, etc.), clasificar como "Nacional"
        else:
            return 'Nacional'
        
    df_unido['Tipo_origen'] = df_unido.apply(clasificar_producto, axis=1)

                                

    # 6. Eliminar productos con clase "SIN CLASE X SUCURSAL" y aquellos que no tienen marca
     
    df_unido = df_unido[~(df_unido['Marca'].isna() | (df_unido['Marca'].str.strip() == ''))]

    df_unido = df_unido[df_unido["Clase"] != "SIN CLASE X SUCURSAL"]

    df_unido["Sucursal"] = df_unido["Sucursal"].replace("PRINCIPAL COTA", "MOSTRADOR COTA") #Correccion bodega 8 esatba como principal cota
    df_unido["Sucursal"] = df_unido["Sucursal"].str.replace("Sucursal ", "", regex=False) #Dejar sin la palabra sucursal




    # 7. Agregar columna con la meta de nivel de servicio según tipo/clase
    metas_nv = {
        'AAA': 0.999,
        'A': 0.98,
        'B': 0.95,
        'C': 0.50
    }

    # Normalizar la columna de clase por si hay espacios o minúsculas
    df_unido["Clase"] = df_unido["Clase"].str.strip().str.upper()

    # Crear columna con la meta
    df_unido["meta_ns"] = df_unido["Clase"].map(metas_nv)

    # 8. Comparar si se cumplió la meta
    # Asegúrate de tener una columna real con el nivel de servicio (ajusta el nombre si es otro)
    if "nivel_servicio" in df_unido.columns:
        df_unido["cumple_meta"] = df_unido["nivel_servicio"] >= df_unido["meta_ns"]




    return df_unido




In [15]:
df_limpio = limpiar_y_transformar(df_unido)
df_limpio = df_limpio[['Sucursal' ,'categoria_producto', 'Marca' , 'Clase', 'Codigo_interno' , 'Septiembre nv', 'Septiembre vp','meta_ns','Tipo_origen','Descripcion']]
df_limpio.head()

,Sucursal,categoria_producto,Marca,Clase,Codigo_interno,Septiembre nv,Septiembre vp,meta_ns,Tipo_origen,Descripcion
0,MOSTRADOR COTA,Filtro,DONSSON,C,025 DA9277,NaN,1.0,0.5,Fabricado,[DAB09277025] DA9277 FILTRO AIRE SEGURIDAD - D...
1,MOSTRADOR COTA,Filtro,DONSSON,C,025 DA8277,1.0,NaN,0.5,Fabricado,[DAB08277025] DA8277 FILTRO AIRE - DONG FENG E...
2,MOSTRADOR COTA,Otros,OTROS,C,136 GS752,1.0,NaN,0.5,Nacional,[DLS00752136] GS752 FILTRO ACEITE - CUMMINS F2...
3,MOSTRADOR COTA,Otros,OTROS,C,136 DAC097,1.0,NaN,0.5,Nacional,[DAC00097136] DAC097 FILTRO AIRE CABINA - VOLK...
4,MOSTRADOR COTA,Filtro,DONSSON,C,025 DA9278,1.0,NaN,0.5,Fabricado,[DAE09278025] DA9278 FILTRO AIRE SEGURIDAD - S...


In [16]:
df_limpio.groupby(['Sucursal','Clase'])['Septiembre vp'].sum()

Sucursal               Clase
MOSTRADOR COTA         A        102.0
                       AAA        6.0
                       B        205.0
                       C        581.0
SUCURSAL BARRANQUILLA  A         43.0
                       AAA        0.0
                       B        106.0
                       C        350.0
SUCURSAL BUCARAMANGA   A         12.0
                       AAA       11.0
                       B         30.0
                       C        176.0
SUCURSAL CALI          A         74.0
                       AAA       33.0
                       B         82.0
                       C        259.0
SUCURSAL CALLE 6       A        151.0
                       AAA       47.0
                       B         60.0
                       C        157.0
SUCURSAL MEDELLIN      A          7.0
                       AAA        0.0
                       B          6.0
                       C        199.0
SUCURSAL NORTE         A         29.0
                     

### Puedo ver las ventas perdidas de todos los meses que han trasncurrido

In [17]:
df_completo = limpiar_y_transformar(df_unido)
df_completo_vp = df_completo[['Sucursal' ,'categoria_producto', 'Marca' , 'Clase', 'Codigo_interno' ,'Enero vp','Febrero vp','Marzo vp','Mayo vp','Abril vp','Junio vp','Julio vp','Agosto vp','Septiembre vp','meta_ns','Tipo_origen','Descripcion']]
df_largo = df_completo_vp.melt(
    id_vars=['Sucursal', 'categoria_producto', 'Marca', 'Clase', 'Codigo_interno', 'meta_ns', 'Tipo_origen', 'Descripcion'],
    value_vars=['Enero vp', 'Febrero vp', 'Marzo vp', 'Abril vp', 'Mayo vp', 'Junio vp', 'Julio vp', 'Agosto vp', 'Septiembre vp'],
    var_name='Mes',
    value_name='Ventas Perdidas'
)


# Quitar el " vp" del final
df_largo['Mes'] = df_largo['Mes'].str.replace(' vp', '', regex=False)

# Diccionario para ordenar meses
orden_meses = {
    'Enero': 1, 'Febrero': 2, 'Marzo': 3, 'Abril': 4,
    'Mayo': 5, 'Junio': 6, 'Julio': 7, 'Agosto': 8,
    'Septiembre': 9, 'Octubre': 10, 'Noviembre': 11, 'Diciembre': 12
}

# Columna auxiliar de orden
df_largo['Mes_num'] = df_largo['Mes'].map(orden_meses)


# Primero eliminar filas que no aportan nada
# Agrupamos por producto y verificamos si TODOS los valores de "Ventas Perdidas" son NaN o 0
mask = df_largo.groupby("Codigo_interno")["Ventas Perdidas"].transform(
    lambda x: x.notna().any() and x.fillna(0).sum() > 0
)

# Filtrar solo los productos que sí tienen al menos una venta perdida
df_filtrado = df_largo[mask]

df_agosto = df_filtrado[df_filtrado["Mes"]=="Agosto"]

df_agosto.to_excel("vp_agosto.xlsx")

# Guardar para BI
df_filtrado.to_excel("v_perdidas/ventas_perdidas_general.xlsx", index=False)




## Unir DF costo

In [18]:
df_inv_costos = df_inventario[['Codigo_interno', 'Sucursal', 'Costo unitario']].drop_duplicates()

df_merged = df_limpio.merge( 
    df_inv_costos,
    on = ['Codigo_interno','Sucursal'], 
    how='left'
)

df_merged['costo_vp'] = df_merged['Septiembre vp'] * df_merged['Costo unitario']

df_limpio= df_merged

In [19]:
df_limpio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38456 entries, 0 to 38455
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Sucursal            38456 non-null  object 
 1   categoria_producto  38456 non-null  object 
 2   Marca               38456 non-null  object 
 3   Clase               38456 non-null  object 
 4   Codigo_interno      38456 non-null  object 
 5   Septiembre nv       37556 non-null  float64
 6   Septiembre vp       1148 non-null   float64
 7   meta_ns             38456 non-null  float64
 8   Tipo_origen         38456 non-null  object 
 9   Descripcion         38456 non-null  object 
 10  Costo unitario      8687 non-null   float64
 11  costo_vp            276 non-null    float64
dtypes: float64(5), object(7)
memory usage: 3.5+ MB


## Ventas perdidas 

In [20]:


# Suponiendo que ya tienes tu df_limpio
# df_limpio = cargar_datos(str(ruta_carpeta))

# 1) Pedir el mes al usuario
mes = input("\nIngrese el mes (ejemplo: julio): ").strip().lower()

# 2) Construir la ruta completa del archivo
nombre_archivo = Path("informes") / f"datos_{mes}.xlsx"

# 3) Guardar el DataFrame en la carpeta 'informes'
df_limpio.to_excel(nombre_archivo, index=False)

print(f"\n✓ Archivo guardado como {nombre_archivo}")



✓ Archivo guardado como informes/datos_.xlsx


In [21]:
vp_general = df_limpio[df_limpio['Septiembre vp']>0]
vp_general.to_excel(f"v_perdidas/ventas_perdidas_{mes}.xlsx",index=False)

# Indicador NS

In [22]:
metas_nv = {
        'AAA': 0.999,
        'A': 0.98,
        'B': 0.95,
        'C': 0.50
    }

def calcular_nivel_servicio(df, mes=None):
    # Paso 1: Identificar columnas de nivel de servicio
    columnas_nv = [col for col in df.columns if col.endswith('nv')]
    
    # Paso 2: Ordenar las columnas por orden real de meses
    orden_meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio',
                   'Julio','Agosto','Septiembre']
    
    columnas_nv_ordenadas = sorted(
        columnas_nv,
        key=lambda x: orden_meses.index(x.split()[0])
    )

    # Paso 3: Si no se especifica el mes, detectar el último con al menos 80% de datos
    if mes is None:
        mes_final = None
        for col in reversed(columnas_nv_ordenadas):
            if df[col].notna().mean() > 0.8:
                mes_final = col
                break
        if mes_final is None:
            raise ValueError("No hay columnas de nivel de servicio con suficientes datos.")
    else:
        mes_final = next((col for col in columnas_nv if col.startswith(mes)), None)
        if mes_final is None:
            raise ValueError(f"No se encontró una columna de nivel de servicio para el mes '{mes}'.")

    # Paso 4: Calcular promedio por sucursal y clase
    resultado = (
        df.groupby(['Sucursal', 'Clase'])[mes_final]
        .mean()
        .reset_index()
        .rename(columns={mes_final: 'Nivel_Servicio'})
    )

    # Paso 5: Evaluar contra la meta según clase
    resultado['Meta'] = resultado['Clase'].map(metas_nv)
    resultado['Cumple'] = resultado['Nivel_Servicio'] >= resultado['Meta']
    resultado['Mes'] = mes_final.split()[0]

    return resultado


In [23]:
# Si quieres que el sistema lo escoja automáticamente:
nivel_servicio = calcular_nivel_servicio(df_limpio)

# O si tú decides el mes (por ejemplo, "Junio"):
nivel_servicio = calcular_nivel_servicio(df_limpio, mes="Septiembre")

nivel_servicio

,Sucursal,Clase,Nivel_Servicio,Meta,Cumple,Mes
0,MOSTRADOR COTA,A,0.973115,0.980,False,Septiembre
1,MOSTRADOR COTA,AAA,0.966250,0.999,False,Septiembre
2,MOSTRADOR COTA,B,0.992600,0.950,True,Septiembre
3,MOSTRADOR COTA,C,0.996927,0.500,True,Septiembre
4,SUCURSAL BARRANQUILLA,A,0.991119,0.980,True,Septiembre
5,SUCURSAL BARRANQUILLA,AAA,1.000000,0.999,True,Septiembre
6,SUCURSAL BARRANQUILLA,B,0.989395,0.950,True,Septiembre
7,SUCURSAL BARRANQUILLA,C,0.998526,0.500,True,Septiembre
8,SUCURSAL BUCARAMANGA,A,0.990273,0.980,True,Septiembre
9,SUCURSAL BUCARAMANGA,AAA,0.985372,0.999,False,Septiembre


In [ ]:
df_largo.head()

,Sucursal,categoria_producto,Marca,Clase,Codigo_interno,meta_ns,Tipo_origen,Descripcion,Mes,Ventas Perdidas,Mes_num
0,MOSTRADOR COTA,Filtro,DONSSON,C,025 DA9277,0.5,Fabricado,[DAB09277025] DA9277 FILTRO AIRE SEGURIDAD - D...,Enero,NaN,1
1,MOSTRADOR COTA,Filtro,DONSSON,C,025 DA8277,0.5,Fabricado,[DAB08277025] DA8277 FILTRO AIRE - DONG FENG E...,Enero,NaN,1
2,MOSTRADOR COTA,Otros,OTROS,C,136 GS752,0.5,Nacional,[DLS00752136] GS752 FILTRO ACEITE - CUMMINS F2...,Enero,NaN,1
3,MOSTRADOR COTA,Otros,OTROS,C,136 DAC097,0.5,Nacional,[DAC00097136] DAC097 FILTRO AIRE CABINA - VOLK...,Enero,NaN,1
4,MOSTRADOR COTA,Filtro,DONSSON,C,025 DA9278,0.5,Fabricado,[DAE09278025] DA9278 FILTRO AIRE SEGURIDAD - S...,Enero,NaN,1


: 